In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
fn = 'points.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))
len(points[points['from_hitchwiki'] == 1.0])

19144

In [3]:
points.head()

,id,lat,lon,rating,country,wait,nickname,comment,datetime,reviewed,banned,ip,dest_lat,dest_lon,signal,ride_datetime,user_id,from_hitchwiki
0,0,40.97,27.51,3.00,TR,NaN,Tamergem,"If you avoid the mini busses, you can get on a...",2011-05-26 10:06:17.000000,1,0,,NaN,NaN,None,None,NaN,1.00
1,1,32.07,34.79,4.00,IL,NaN,None,None,None,1,0,,NaN,NaN,None,None,NaN,NaN
2,2,41.73,27.22,4.00,TR,NaN,None,None,None,1,0,,NaN,NaN,None,None,NaN,NaN
3,3,41.10,29.01,3.00,TR,NaN,Xavierallard,There is a lot of traffic there and little spa...,2011-03-15 12:52:11.000000,1,0,,NaN,NaN,None,None,NaN,1.00
4,4,30.17,67.00,3.00,PK,NaN,None,None,None,1,0,,NaN,NaN,None,None,NaN,NaN


In [4]:
hitchwiki_users = points[points['from_hitchwiki'] == 1.0].nickname.unique()
len(hitchwiki_users)

2562

In [5]:
hitchwiki_users

array(['Tamergem', 'Xavierallard', 'Rozwal', ..., 'Lunasis', 'Francagini',
       'Silverness'], shape=(2562,), dtype=object)

In [ ]:
# test on osm wiki
import requests

S = requests.Session()

URL = "https://wiki.openstreetmap.org/api.php"

def get_wiki_user(name):
    try:
        PARAMS = {
            "action": "query",
            "format": "json",
            "list": "users",
            "ususers": name,
            "usprop": "blockinfo|groups|editcount|registration|emailable|gender"
        }

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()

        return DATA["query"]["users"][0]["gender"]
    except Exception as e:
        print(f"Error fetching data for user {name}: {e}")
        return None
    
get_wiki_user("Tillwenke")

'male'

In [3]:
import requests

S = requests.Session()

URL = "https://hitchwiki.org/en/api.php"

def get_wiki_user(name):
    try:
        PARAMS = {
            "action": "query",
            "format": "json",
            "list": "users",
            "ususers": name,
            "usprop": "blockinfo|groups|editcount|registration|emailable|gender"
        }

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()

        return DATA["query"]["users"][0]["gender"]
    except Exception as e:
        print(f"Error fetching data for user {name}: {e}")
        return None

genders = []

for u in tqdm(hitchwiki_users):
    genders.append((u, get_wiki_user(u)))

NameError: name 'hitchwiki_users' is not defined

In [7]:
users = pd.DataFrame(genders, columns=['nickname', "gender"])
users.gender.value_counts()

gender
unknown    1940
male        224
female       48
Name: count, dtype: int64

In [12]:
users.to_csv('hitchwiki_users.csv', index=False)

In [8]:
def match_gender(row):
    if row["from_hitchwiki"] == 1.0:
        gender = users.loc[users['nickname'] == row['nickname']]["gender"].iloc[0]
        if gender == "unknown":
            return None
        return gender
    return None

In [9]:
points["gender"] = points.apply(match_gender, axis=1)

In [10]:
len(points[points["gender"].notna()])

3359

In [11]:
points.gender.value_counts()

gender
male      2879
female     480
Name: count, dtype: int64